### OMP(Orthogoanl Matching Pursuit, compressed sensing algorithm) based channel estimation & infinite/limited rate feedback

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import scipy.linalg as sci
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Parameter 초기화

In [ ]:
## Learning parameters
initial_run = 1 #1: starts training from scratch; 0: resumes training 
n_epochs = 5000 #Number of training epochs, for observing the current performance set it to 0
learning_rate = 0.0005 #Learning rate

batch_size = 1024 #Mini-batch size
test_size = 10000 #Size of the validation/test set
batch_per_epoch = 20 #Numbers of mini-batches per epoch

anneal_param = 1.0 #Initial annealing parmeter
annealing_rate = 1.001 #Annealing rate

In [4]:
## System parameters
M = 64 #Number of BS antennas
P = 1 #Power
K =  2 #Number of users
L = 8 #Number of pilots
Lp = 2 #Number of paths
B = 30 #Number of feedback bits per user

## Limited scattering channel parameters
LSF_UE = np.array([0.0,0.0],dtype=np.float32) #Mean of path gains for K users
Mainlobe_UE= np.array([0,0],dtype=np.float32) #Center of the AoD range for K users
HalfBW_UE = np.array([30.0,30.0],dtype=np.float32) #Half of the AoD range for K users

# SNR
snr_dl = 10 #SNR in dB
noise_std_dl = np.float32(np.sqrt(1/2)*np.sqrt(P/10**(snr_dl/10))) #STD of the Gaussian noise (per real dim.)

In [5]:
print(HalfBW_UE)

[30. 30.]


### OMP based sparse channel estimation

#### **Input**
1. Pilot sequence X , DFT matrix of (-30, 30), 1000 intervals S -> measurement matrix Phi=transpose(X)*transpose(S)=transpose(S*X)
2. First residual/data vector u=transpose(y)=[y_1, ..., y_L]=(h_k^H)*[x_1, ..., x_L]+[w_1, ..., w_L]=(h_k^H)*S * transpose(X)*[x_1, ..., x_L]+[w_1, ..., w_L]

In [ ]:
def OMPce(h_act, pilot_sequence, num_users, Mainlobe_UE, HalfBW_UE, noise_std, sparsity_level, dict_size):
    AoD_low = Mainlobe_UE[0]-HalfBW_UE[0]
    AoD_high = Mainlobe_UE[0]+HalfBW_UE[0]
    S = torch.zeros([dict_size, len(pilot_sequence[:, 0])])
    
    for ss in range(dict_size):
        S[ss, :]=torch.exp(1j*torch.pi*[m for m in range(pilot_sequence[:, 0])]*(AoD_low + (AoD_high-AoD_low) * ss/dict_size))
    
    for hh in range(h_act[0]):
        H = h_act[hh, :, :].squeeze(0)
        Y = H * pilot_sequence + torch.randn([num_users, len(pilot_sequence[0, :])])
        Hhat = torch.zeros([num_users, len(pilot_sequence[0, :])])
        
        for kk in range(num_users):
            u = Y[kk, :].T
            Phi = pilot_sequence.T * S.T
            idx = torch.zeros(sparsity_level, 1)
            v = u
            
            for tt in range(sparsity_level):
                idx[tt] = torch.argmax(torch.abs(Phi.T * v))
                phi_s = [Phi[:, i] for i in idx[:tt]]
                b = sci.pinv(phi_s)*u
                v = v - phi_s * B
                
            hhat = b.T * [S[i, :] for i in idx]
            Hhat[kk, :] = hhat
            
    return Hhat